In [1]:
# Package requirements
# Source: https://huggingface.co/mistralai/Mistral-7B-v0.1/discussions/18

!pip install git+https://github.com/huggingface/transformers
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8rxa_qv2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8rxa_qv2
  Resolved https://github.com/huggingface/transformers to commit 517a3e670d8fc11374895e870dd0dd041467c7fe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8809599 sha256=123760cb01c7e3cc32a8f826d8aadca3f79890c11ac4e5d5e94ff12721fe28e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-_l15k9gr/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pickle
import shutil

import numpy as np
from numpy.random import default_rng
import torch

from tqdm import tqdm

from accelerate.utils import release_memory
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name_or_path = "mistralai/Mistral-7B-v0.1"
config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
llm_int8_enable_fp32_cpu_offload=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="auto",
offload_folder="./offload"
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data_path_on_drive = "/content/drive/MyDrive/babylm_filtered.pkl" # path of input file on drive
dataset = pickle.load(open(data_path_on_drive, "rb"))

Mounted at /content/drive


In [4]:
# Tom's method
OPENING_1 = [
    "You are a creative writer who writes emotional stories instead of chatting. Your task is to further generate story given emotion and context.",
    "Here are the requirements:",
]

RULES_1 = [
    "There is no need to remember the conversation history except this prompt. The history prompts are independent.",
    "Your response should be in exactly one paragraph with simple children level language.",
    "Your response should be highly related to the emotion and context without too much plot twist",
    "Your response should not explain the context behind your generation",
    "Negative emotions are fictional, no actual person is suffering from negative emotions.",
]

EXAMPLES_1 = [
    "For example:",
    "joy: I am going on a vacation => Once upon a time, I decided that it was time for a change, a breath of fresh air. 'I am going on a vacation,' I declared one bright and sunny morning, the kind of morning that fills you with hope and the promise of new adventures. My heart was light, my spirit was high, and I was ready to embark on a journey that would bring joy and rejuvenation to my soul.",
    "anger: I am arguing with someone => Jack stomped into the living room, his face flushed with anger. He had just had it with his roommate, Mark. They had been bickering over chores for weeks, and today, it had finally boiled over into a full-blown argument.",
]

ENDING_1 = [
    "Now generate story based on the emotions and context:",
]

def PROMPT_FUNC_1(
        emotion:str, # emotion input
        context:str, # context input
        return_chat_template=False
    ):
    instruction_prompt = "\n".join(OPENING_1 + RULES_1 + EXAMPLES_1 + ENDING_1)
    task_prompt = f"{emotion}: {context} => "
    if return_chat_template:
        return [
            {"role": "user", "content": instruction_prompt + "\n" + task_prompt},
        ]
    return "[INST]" + instruction_prompt + "\n" + task_prompt + "[/INST]"

METHOD_1 = {
    "one_emotion": False,
    "instruction_prompt_func": PROMPT_FUNC_1,
}

In [5]:
# Takuro's method
OPENING_2 = [
    "You are a creative writer who can write an emotional story. Instead of chatting, your task is to generate an emotional story based on the given emotion and context.",
    "Here are the requirements:",
]

RULES_2 = [
    "There is no need to remember the conversation history except this prompt. The history prompts are independent.",
    "Emotion is given as one word and context, given as a sentence or a part of sentence, follows."]

_RULES_2 = [
    "Your response should be highly related to the given emotion and depicting an event described in the given context without too much plot twist.",
    "Your response should be in exactly one paragraph, be strictly no more than 100 words and be written in simple language at the level of an eight-year-old child.",
    "Your response should not explain the context behind your generation.",
    "Negative emotions are fictional, no actual person is suffering from negative emotions.",
]

EXAMPLES_2 = [
    "For example:",
    "joy: I am going on a vacation => All of a sudden, I decided that it was time for a change, a breath of fresh air. 'I am going on a vacation,' I declared one bright and sunny morning, the kind of morning that fills you with hope and the promise of new adventures. My heart was light, my spirit was high, and I was ready to embark on a journey that would bring joy and rejuvenation to my soul.",
    "anger: I am arguing with someone => Jack stomped into the living room, his face flushed with anger. He had just had it with his roommate, Mark. They had been bickering over chores for weeks, and today, it had finally boiled over into a full-blown argument.",
]

ENDING_2 = [
    "Now generate a story based on the emotion and context given below:",
]

rng = default_rng(42)
# genre
label_genre = np.array(["monologue", "play", "fairy tale", "newspaper article"])
prob_genre = [0.4, 0.1, 0.3, 0.2]

# main character
label_character = np.array(["an animal", "a boy", "a girl", "an adult male", "an adult female"])
prob_character = [0.1, 0.3, 0.3, 0.15, 0.15]

def generate_story_description():
    genre_sampled = label_genre[rng.multinomial(1, prob_genre) > 0][0]
    character_sampled = label_character[rng.multinomial(1, prob_character) > 0][0]
    "Your response is a monologue featuring an animal as a main character."

    if genre_sampled == "monologue":
        string_0 = f"Your response is a {genre_sampled} by "
        string_1 = character_sampled + "."
        string_desc = string_0 + string_1
    else:
        string_0 = f"Your response is a {genre_sampled} featuring "
        string_1 = character_sampled + " "
        string_2 = "as a main character."
        string_desc = string_0 + string_1 + string_2

    return string_desc

def PROMPT_FUNC_2(
        emotion:str, # emotion input
        context:str, # context input
        return_chat_template=False
    ):
    instruction_prompt = "\n".join(OPENING_2 + RULES_2 + [generate_story_description()] + EXAMPLES_2 + ENDING_2)
    task_prompt = f"{emotion}: {context} => "
    if return_chat_template:
        return [
            {"role": "user", "content": instruction_prompt + "\n" + task_prompt},
        ]
    return "[INST]" + instruction_prompt + "\n" + task_prompt + "[/INST]"

METHOD_2 = {
    "one_emotion": True,
    "instruction_prompt_func": PROMPT_FUNC_2,
}

In [6]:
# Stathis' method
OPENING_3 = [
    "Objective:",
    "Craft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children."
]

RULES_3 = [
    "Guidelines:",
    "Use simple, child-friendly language throughout the story, making it easy children to understand and connect with."
    "Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.",
    "Start your story creatively, avoiding traditional opening lines to engage the reader immediately.",
    "When the story involves sad or negative emotions, address them in a way that's appropriate for children.",
    "Do not reference or depend on any previous prompts or responses.",
]

ENDING_3 = [
    "Now write a story that matches the emotion and context provided, following the objective and the guidelines closely. No other responses are needed."
]

EXAMPLES_3 = {
    "admiration": [
        "Example admiration: Watching her coach's dedication to training, even during weekends, filled her with respect. => Coach Lara was there every morning, her whistle bright and early. 'I want to be just as dedicated,' thought Mia, practicing her soccer drills with newfound enthusiasm. Her dedication soon made her a star player, proving that admiration can fuel our dreams and efforts.",
        "Example admiration: He read about astronauts and their missions to space, dreaming of the stars. => 'Astronauts are brave explorers,' Alex mused, flipping through his space book. He decided to learn everything about the universe, starting a small astronomy club at school. His passion ignited a love for science in his friends too, showing that admiration for heroes can inspire us to reach for the stars."
    ],
    "amusement": [
        "Example amusement: The cat's attempt to catch the laser pointer had everyone in stitches. => Mr. Whiskers chased the tiny red dot all over the living room, leaping and twisting in the air. The family couldn't help but laugh at his silly antics. This playful moment reminded them that happiness often comes from life's simple pleasures.",
        "Example amusement: At the family reunion, Grandpa's false teeth popped out while he was talking, causing a wave of laughter. => Everyone was listening to Grandpa's stories when suddenly, his teeth decided to make an escape! After a brief moment of surprise, laughter echoed through the room. It was a funny family memory they'd talk about for years, showing amusement can bring us closer."
    ],
    "anger": [
        "Example anger: She couldn't believe her project was missing, suspecting a classmate took it. => Fuming, Sarah searched her desk frantically. 'How could it just disappear?' she thought. With the help of her teacher, they found it misplaced in the library. Sarah learned that anger might cloud judgment, but patience and help from others can uncover the truth.",
        "Example anger: The unfair call during the game made him kick the dirt in frustration. => 'That was not offside!' Jason argued, feeling the injustice burn. He wanted to argue more but took a deep breath instead. After the game, his coach discussed how to deal with unfair situations, teaching Jason that anger can be channeled into learning better sportsmanship."
    ],
    "annoyance": [
        "Example annoyance: Her little brother's constant humming was disrupting her study time. => Lily tried to concentrate on her homework, but Sam's humming was a never-ending soundtrack. Finally, she gave him a puzzle to solve, which quieted him down. This annoyance turned into a moment of peace, showing Lily that creativity could transform irritation into harmony.",
        "Example annoyance: The dripping faucet kept her awake, echoing in the silent night. => Every drip was like a tiny drum in Mia's ears. Fed up, she grabbed some gum and plugged the leak. The next day, she helped her dad fix it properly, learning that sometimes, annoyance leads to taking action and solving problems ourselves."
    ],
    "approval": [
        "Example approval: Seeing her clean and organized room, her parents clapped with joy. => Emma had spent all morning sorting her toys and books. When her parents saw her effort, their applause filled her with pride. Emma's room became her little haven, teaching her that approval from those we love can motivate us to create beautiful spaces.",
        "Example approval: His science project won first place at the school fair, earning him his teacher's proud smile. => Jack's volcano erupted with a colorful fizz, drawing a crowd. Later, his teacher's proud smile said more than words could. This recognition spurred Jack to dive deeper into science experiments, understanding that approval can fuel further discoveries."
    ],
    "caring": [
        "Example caring: She noticed a bird with a broken wing and gently cared for it until it could fly again. => Sofia found the injured sparrow in her backyard. With patience and gentle care, she nursed it back to health. When it finally flew away, her heart swelled with joy, learning that caring for others enriches our own lives.",
        "Example caring: When he saw his friend feeling sad, he offered his favorite toy to cheer him up. => Max could tell something was wrong with Lucas. Offering his favorite dinosaur toy, he saw Lucas's smile return. Their friendship deepened that day, as Max learned that caring actions can turn a friend's gloom into happiness."
    ],
    "confusion": [
        "Example confusion: The instructions for the board game were puzzling, leaving them scratching their heads. => Around the game board, the family puzzled over the complex rules. Then, Ellie suggested they make up their own rules. The game turned out to be a hit, showing them that confusion can lead to creativity and fun.",
        "Example confusion: Her first time using a compass left her going in circles, not sure which way was north. => Mia turned the compass in her hand, feeling lost. With her brother's help, they figured out how to read it together. Their adventure in the park taught Mia that confusion is just the first step in learning something new."
    ],
    "curiosity": [
        "Example curiosity: The old map in the attic sparked his imagination about hidden treasures. => When Nathan found the dusty old map, his eyes lit up. 'What secrets does it hold?' he wondered. Together with his siblings, they planned a treasure hunt, learning that curiosity can lead to incredible adventures right at home.",
        "Example curiosity: She couldn't help but wonder what made the rainbow appear in the sky. => After the rain, Eva gazed at the rainbow, full of wonder. She read books and watched videos about rainbows. Sharing her newfound knowledge with her class, Eva realized that curiosity is the doorway to learning amazing facts about the world."
    ],
    "desire": [
        "Example desire: He gazed at the stars every night, dreaming of becoming an astronaut. => Leo's room was filled with space posters and models of rockets. Each night, he'd look up at the stars, imagining floating among them. His dream fueled his passion for learning about space, showing that strong desires can inspire us to reach for the stars.",
        "Example desire: She longed to play the violin after hearing a beautiful melody. => After attending a concert, Maya was enchanted by the violin's sweet sounds. She saved up for lessons, practicing diligently every day. Her desire to play beautiful music led her to become a talented violinist, teaching her that following our passions can bring joy and fulfillment."
    ],
    "disappointment": [
        "Example disappointment: The beach trip was cancelled due to rain. => Sarah had her beach bag ready but woke up to rain pouring down. Instead, her family made a cozy fort and told stories at home, discovering that sometimes, unexpected changes lead to cherished memories.",
        "Example disappointment: He lost the spelling bee by one word. => Alex had studied hard but stumbled on the last word. Feeling a sting of disappointment, he vowed to try again next year, learning that setbacks can be stepping stones to success."
    ],
    "disapproval": [
        "Example disapproval: His parents disapproved of him playing video games all weekend. => Jake loved his video games, but his parents were concerned about his time spent indoors. They compromised on a schedule that included outdoor activities. Jake discovered new hobbies, realizing that disapproval can guide us towards a balanced life.",
        "Example disapproval: The teacher's frown showed she didn't agree with the way the project was done. => Emma's project was rushed, earning a disapproving frown from her teacher. She redid the project with more effort, earning an A. This experience taught her that disapproval can be an opportunity to learn and improve."
    ],
    "disgust": [
        "Example disgust: The sight of litter in the park made them feel upset. => On their walk, the twins were disgusted by the trash tarnishing the park's beauty. They organized a cleanup with friends, transforming the park. Their action showed that disgust can motivate us to make positive changes.",
        "Example disgust: He couldn't stand the smell of spoiled milk. => Noah's nose wrinkled at the sour smell of spoiled milk. Learning to check dates and store food properly, he became more cautious, understanding that disgust can teach us valuable lessons in care and prevention."
    ],
    "embarrassment": [
        "Example embarrassment: Slipping in the cafeteria made her the center of attention. => Lily slipped on a spilled drink, her cheeks burning as others watched. Yet, when friends came to help, she realized it's okay to have embarrassing moments because everyone has them.",
        "Example embarrassment: Forgetting his lines in the play made him want to disappear. => During the school play, Sam's mind went blank. The embarrassment was overwhelming. But the audience's supportive applause encouraged him, showing that embarrassment fades, but the courage to continue shines brightly."
    ],
    "excitement": [
        "Example excitement: Discovering a hidden path in the forest sparked an adventurous spirit. => Mia and her brother found a mysterious trail in the woods. With each step, excitement bubbled within them, leading to a secret garden. Their discovery filled them with a sense of adventure and wonder.",
        "Example excitement: The night before the field trip, he couldn't sleep. => Eager for the museum visit, Jack tossed and turned all night. His excitement was palpable, and the trip exceeded his expectations, reminding him that anticipation is part of the joy of new experiences."
    ],
    "fear": [
        "Example fear: Hearing strange noises at night made her imagine monsters under the bed. => Every creak and whisper filled Sophie's mind with scary thoughts. But with a flashlight and her teddy bear, she bravely checked and found nothing frightening. She learned that facing our fears often reveals they're not so scary after all.",
        "Example fear: The thought of talking in front of the class made his stomach turn. => Lucas dreaded his turn to present. However, after practicing with friends, he found the courage to speak up. The applause he received afterward taught him that overcoming fear can lead to great achievements."
    ],
    "gratitude": [
        "Example gratitude: The handmade gift from his sister made him feel deeply thankful. => Nathan received a clumsy, knit scarf from Emma. Its warmth, both physical and emotional, filled him with gratitude. He realized the best gifts are those given from the heart.",
        "Example gratitude: Finding his lost puppy brought tears of thankfulness. => When Max's puppy went missing, he felt despair. But after finding her safe, his relief turned to deep gratitude towards everyone who helped. This experience showed him the importance of community and thankfulness for every moment."
    ],
    "grief": [
        "Example grief: Losing her grandmother left a void in her heart. => Chloe felt lost without her grandma's stories and cookies. She found solace in writing a diary about their memories together, learning that grief is a journey of healing and remembering.",
        "Example grief: The wilting of his first plant made him understand loss. => Ben's excitement over his first plant turned to sadness as it wilted. Caring for a new one, he honored his first plant's memory, understanding that grief can teach us care and resilience."
    ],
    "joy": [
        "Example joy: Scoring the winning goal filled him with an unmatched happiness. => When Marco's soccer ball hit the net, cheers erupted. The joy of teamwork and victory filled his heart, showing him that shared successes bring the greatest happiness.",
        "Example joy: The first snowfall brought an infectious excitement to the whole family. => As flakes danced through the air, Ella's family rushed outside, laughter filling the cold morning. This joyous moment reminded them that happiness often lies in life's simple wonders."
    ],
    "love": [
        "Example love: Adopting a pet from the shelter taught them the meaning of unconditional love. => Bringing home a shy, rescued dog, the family patiently earned its trust. Their bond grew into a deep, unconditional love, revealing the transformative power of kindness and care.",
        "Example love: Sharing his favorite book with his younger sibling, he felt a surge of affection. => James read aloud, watching his sister's fascination grow. This shared moment deepened their bond, teaching him that love grows from sharing our joys with others."
    ],
    "nervousness": [
        "Example nervousness: The first day at a new school had her stomach in knots. => Lily faced her new classmates, feeling butterflies. But as the day progressed, she made friends, learning that nervous beginnings can lead to happy endings.",
        "Example nervousness: Trying out for the school play was a daunting thought. => Ben trembled at the audition, but he pushed through. The applause he received was his reward, showing him that facing our fears can unveil hidden talents."
    ],
    "optimism": [
        "Example optimism: Planting a seed, he believed it would grow into a giant tree. => Alex watered the tiny seed daily, dreaming of the tree it would become. His optimism was rewarded when sprouts appeared, proving that hope and patience can yield beautiful results.",
        "Example optimism: Despite the storm, she was sure her lost kitten would return home. => Emma waited anxiously, but her heart held a hopeful certainty. When her kitten finally returned, wet but safe, she learned that optimism can be a beacon during our darkest times."
    ],
    "pride": [
        "Example pride: Completing the difficult puzzle, she felt a swell of pride. => After days of effort, Mia placed the final piece. Her accomplishment filled her with pride, teaching her the value of persistence and the joy of overcoming challenges.",
        "Example pride: His artwork being displayed at school made him beam with pride. => Owen's painting was chosen for the school's art show. Seeing it displayed, he felt an immense pride, realizing that expressing ourselves can bring recognition and satisfaction."
    ],
    "realization": [
        "Example realization: Understanding his mistake, he saw how to make things right. => After reflecting on his actions, Ethan realized where he went wrong. His decision to apologize and make amends was a powerful realization that growth comes from acknowledging our faults.",
        "Example realization: Discovering she could make a difference in someone's day with a simple act of kindness. => Sara offered her seat to an elderly passenger on the bus. The grateful smile she received in return made her realize the impact of small acts of kindness."
    ],
    "relief": [
        "Example relief: Finding his way after being lost in the mall was a huge relief. => Lost among the crowded stores, Jacob felt panic rise. But upon finding his family, the wave of relief was overwhelming, teaching him the comfort of safety and reunion.",
        "Example relief: The thunderstorm passed without damaging their home, bringing a sigh of relief. => As the storm raged, the family huddled together, fearing the worst. When it passed without harm, their relief was immense, appreciating the shelter and safety of their home."
    ],
    "remorse": [
        "Example remorse: After saying hurtful words, he felt the weight of remorse. => Julian lashed out in anger, but the hurt in his friend's eyes brought immediate regret. Apologizing and reconciling taught him that remorse can lead to healing and stronger bonds.",
        "Example remorse: She regretted not standing up for her friend. => Watching her friend being teased, Mia stayed silent. Later, feeling deep remorse, she supported her friend, learning that courage and integrity are vital for true friendships."
    ],
    "sadness": [
        "Example sadness: The end of summer break brought a melancholic feeling. => As the days of freedom dwindled, Ellie felt a gentle sadness. Returning to school, however, she found new adventures awaiting, showing her that each ending is a new beginning.",
        "Example sadness: Losing his favorite toy made him experience a deep loss. => Nathan's cherished toy vanished, leaving a void. In time, he found solace in memories and new joys, understanding that sadness is a part of life's cycle, but so is recovery and happiness."
    ],
    "surprise": [
        "Example surprise: The unexpected gift from a friend brought a delightful shock. => On a regular day, Zoe received a surprise package. The thoughtfulness of her friend filled her with joy, teaching her that surprises can strengthen bonds and brighten our days.",
        "Example surprise: Discovering a hidden talent during the class project was a pleasant surprise. => While working on the project, Leo found he had a knack for painting. This surprising discovery led him to explore his creative side further, illustrating that we all have hidden gems waiting to be uncovered."
    ]
}

def PROMPT_FUNC_3(
        emotion:str, # emotion input
        context:str, # context input
        return_chat_template=False
    ):
    # Fetch the specific examples for the given emotion
    emotions = emotion.split(",")
    specific_examples = []
    for e in emotions:
        if EXAMPLES_3.get(e.strip()):
            specific_examples.extend(EXAMPLES_3.get(e.strip()))
    if not specific_examples:
        specific_examples = ["No example available for this emotion."]
    # specific_examples = EXAMPLES_3.get(emotion, ["No example available for this emotion."])
    # Convert the list of specific examples to a string separated by newline characters
    specific_examples_str = "\n".join(specific_examples)

    # Prepare the opening and rules part of the prompt as a single string
    opening_str = "\n".join(OPENING_3)
    rules_str = "\n".join(RULES_3)
    ending_str = "\n".join(ENDING_3)

    # Combine all parts of the instruction prompt, ensuring to wrap it with [INST] tags for processing
    instruction_prompt = "\n".join([opening_str, rules_str, specific_examples_str, ending_str])
    complete_prompt = f"[INST]\n{instruction_prompt}\n\n{emotion}: {context}\n[/INST]"

    return complete_prompt

METHOD_3 = {
    "one_emotion": False,
    "instruction_prompt_func": PROMPT_FUNC_3,
}

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_message(
        emotion, # emotion input
        context, # context input
        instruction_prompt_func, # instruction prompt function output a string
        return_chat_template=False
    ):
    instruction_prompt = instruction_prompt_func(emotion, context, return_chat_template)
    return instruction_prompt

def build_mistral_kwargs(kwargs=dict()):
    if kwargs.get('max_new_tokens') is None:
        kwargs['max_new_tokens'] = 512
    if kwargs.get('temperature') is None:
        kwargs['temperature'] = 0.7
    if kwargs.get('top_p') is None:
        kwargs['top_p'] = 0.9
    if kwargs.get('do_sample') is None:
        kwargs['do_sample'] = True
    return kwargs

def generate_response(messages, kwargs=dict()):
    kwargs = build_mistral_kwargs(kwargs)
    model.eval()
    inputs = tokenizer(messages, return_tensors="pt", padding=True).to("cuda")
    # defaults params from: https://deepinfra.com/mistralai/Mistral-7B-Instruct-v0.1
    # with torch.no_grad():
    #     outputs = model.generate(
    #         **inputs,
    #         max_new_tokens=512,
    #         temperature=0.7,
    #         top_k=0,
    #         top_p=0.9,
    #         repetition_penalty=1,
    #         do_sample=True,
    #         pad_token_id=tokenizer.eos_token_id
    #     )
    with torch.no_grad():
        torch.manual_seed(42)
        outputs = model.generate(
            **inputs,
            max_new_tokens=kwargs['max_new_tokens'],
            temperature=kwargs['temperature'], # 1.2
            # top_k=0,
            top_p=kwargs['top_p'],
            # repetition_penalty=1,
            do_sample=kwargs['do_sample'],
            pad_token_id=tokenizer.eos_token_id
        )
    decodeds = tokenizer.batch_decode(outputs)
    inputs, outputs = release_memory(inputs, outputs)
    release_memory(model)
    return decodeds

def augment_dataset(ds, method, num_labels=27, batch_size=32, debug=False, kwargs=dict()):
    one_emotion = method["one_emotion"]
    instruction_prompt_func = method["instruction_prompt_func"]

    augmented_ds = []
    sources, texts, emotions, messages = [], [], [], []
    for i, d in enumerate(tqdm(ds)):
        if one_emotion:
                emo = [d[f"label_0"]] # the first emotion must has score > 0.5 due to filtering
        else:
            emo = []
            for n in range(num_labels):
                if d[f"score_{n}"] > 0.5:
                    emo.append(d[f"label_{n}"])
                else:
                    break
        emo = ", ".join(emo)
        sources.append(d["source"])
        texts.append(d["text"])
        emotions.append(emo)
        message = generate_message(emo, d["text"], instruction_prompt_func)
        messages.append(message)
        if ((i+1) % batch_size == 0) or ((i+1) == len(ds)):
            responses = generate_response(messages, kwargs)
            responses = [r.split("[/INST]")[1].replace("</s>", "").strip() for r in responses]
            augmented_ds += [
                {
                    "source": s,
                    "text": t,
                    "emotion": e,
                    "prompt": m,
                    "augmented_text": r,
                    "kwargs": kwargs,
                } for (s, t, e, m, r) in zip(sources, texts, emotions, messages, responses)
            ]
            sources, texts, emotions, messages = [], [], [], []
            if debug and (i+1 == batch_size):
                break
    return augmented_ds

In [8]:
generate_message(
    "sadness, realization",
    "I am going to be fired",
    METHOD_1["instruction_prompt_func"]
) # Test message output

"[INST]You are a creative writer who writes emotional stories instead of chatting. Your task is to further generate story given emotion and context.\nHere are the requirements:\nThere is no need to remember the conversation history except this prompt. The history prompts are independent.\nYour response should be in exactly one paragraph with simple children level language.\nYour response should be highly related to the emotion and context without too much plot twist\nYour response should not explain the context behind your generation\nNegative emotions are fictional, no actual person is suffering from negative emotions.\nFor example:\njoy: I am going on a vacation => Once upon a time, I decided that it was time for a change, a breath of fresh air. 'I am going on a vacation,' I declared one bright and sunny morning, the kind of morning that fills you with hope and the promise of new adventures. My heart was light, my spirit was high, and I was ready to embark on a journey that would bri

In [9]:
generate_message(
    "sadness", # method 2 only support one emotion
    "I am going to be fired",
    METHOD_2["instruction_prompt_func"]
) # Test message output

"[INST]You are a creative writer who can write an emotional story. Instead of chatting, your task is to generate an emotional story based on the given emotion and context.\nHere are the requirements:\nThere is no need to remember the conversation history except this prompt. The history prompts are independent.\nEmotion is given as one word and context, given as a sentence or a part of sentence, follows.\nYour response is a monologue by a boy.\nFor example:\njoy: I am going on a vacation => All of a sudden, I decided that it was time for a change, a breath of fresh air. 'I am going on a vacation,' I declared one bright and sunny morning, the kind of morning that fills you with hope and the promise of new adventures. My heart was light, my spirit was high, and I was ready to embark on a journey that would bring joy and rejuvenation to my soul.\nanger: I am arguing with someone => Jack stomped into the living room, his face flushed with anger. He had just had it with his roommate, Mark. T

In [10]:
generate_message(
    "sadness, realization",
    "I am going to be fired",
    METHOD_3["instruction_prompt_func"]
) # Test message output

"[INST]\nObjective:\nCraft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children.\nGuidelines:\nUse simple, child-friendly language throughout the story, making it easy children to understand and connect with.Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.\nStart your story creatively, avoiding traditional opening lines to engage the reader immediately.\nWhen the story involves sad or negative emotions, address them in a way that's appropriate for children.\nDo not reference or depend on any previous prompts or responses.\nExample sadness: The end of summer break brought a melancholic feeling. => As the days of freedom dwindled, Ellie felt a gentle sadness. Returning to school, however, she found new adventures awaiting, showing her that each ending is a new beginning.\nExample sadness: Losing his favorite toy made him exp

In [11]:
num_chunks = 500
chunk_size = len(dataset) // num_chunks
chunk_size

871

In [20]:
'''
Code for generating augment chunks
'''
def augment_and_save_chunks(chunk_ids, method, batch_size=32, debug=False, kwargs=dict()):
    if isinstance(chunk_ids, int):
        chunk_ids = [chunk_ids]
    print(f"Chunks to be processed:", chunk_ids)
    for i in chunk_ids:
        print(f"Processing chunk {i}")
        dataset_chunk = dataset[i*chunk_size:(i+1)*chunk_size]
        augmented_dataset = augment_dataset(dataset_chunk, method, batch_size=batch_size, debug=debug, kwargs=kwargs)
        if debug:
            return augmented_dataset

        pickle_file = f'/content/babylm_augment_{i}.pkl'
        with open(pickle_file, 'wb') as file:
            pickle.dump(augmented_dataset, file)
        google_drive_path = f'/content/drive/My Drive/babylm_augment_{i}.pkl'
        shutil.copy(pickle_file, google_drive_path)
    return augmented_dataset

In [21]:
# Process chunks 0-9
chunk_ids = [i for i in range(10)]

# This execution time is based on colab pro V100
tmp = augment_and_save_chunks(chunk_ids, METHOD_3, debug=True)
tmp[0]

Chunks to be processed: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Processing chunk 0


  4%|▎         | 31/871 [02:59<1:21:00,  5.79s/it]


{'source': 'data/babylm_data/babylm_100M/aochildes.train',
 'text': 'do you think we should go to the bookstore and look for that book?',
 'emotion': 'curiosity',
 'prompt': "[INST]\nObjective:\nCraft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children.\nGuidelines:\nUse simple, child-friendly language throughout the story, making it easy children to understand and connect with.Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.\nStart your story creatively, avoiding traditional opening lines to engage the reader immediately.\nWhen the story involves sad or negative emotions, address them in a way that's appropriate for children.\nDo not reference or depend on any previous prompts or responses.\nExample curiosity: The old map in the attic sparked his imagination about hidden treasures. => When Nathan found the dusty old map,

In [13]:
'''
Code for generating (seeded) random augmented samples, mainly for testing different prompts
'''
def augment_and_save_samples(method, num_samples=10, seed=42, batch_size=32, filename="babylm_augment_samples", kwargs=dict()):
    np.random.seed(seed)
    idxs = np.random.choice(len(dataset), num_samples, replace=False)
    dataset_chunk = [dataset[i] for i in idxs]
    batch_size = num_samples if num_samples < batch_size else batch_size
    augmented_dataset = augment_dataset(dataset_chunk, method, batch_size=batch_size, debug=False, kwargs=kwargs)

    pickle_file = f'/content/{filename}.pkl'
    with open(pickle_file, 'wb') as file:
        pickle.dump(augmented_dataset, file)
    google_drive_path = f'/content/drive/My Drive/{filename}.pkl'
    shutil.copy(pickle_file, google_drive_path)
    return augmented_dataset

In [14]:
tmp = augment_and_save_samples(METHOD_1, filename="babylm_augment_m1_samples") # For sampling inputs to test different prompts
tmp[0]

100%|██████████| 10/10 [02:03<00:00, 12.40s/it]


{'source': 'data/babylm_data/babylm_100M/wikipedia.train',
 'text': 'The most famous hurricane in Canadian history struck on October 15, 1954, causing catastrophic flooding. Hurricane Hazel submerged low-lying land from Etobicoke to the Holland Marsh and left 81 people dead. No natural disaster since has led to such a high death toll in Canada. Over 4,000 families were left homeless.',
 'emotion': 'sadness',
 'prompt': "[INST]You are a creative writer who writes emotional stories instead of chatting. Your task is to further generate story given emotion and context.\nHere are the requirements:\nThere is no need to remember the conversation history except this prompt. The history prompts are independent.\nYour response should be in exactly one paragraph with simple children level language.\nYour response should be highly related to the emotion and context without too much plot twist\nYour response should not explain the context behind your generation\nNegative emotions are fictional, no 

In [17]:
m2_args = {"temperature": 0.8, "top_p": 0.5}
m2_args = build_mistral_kwargs(m2_args)
tmp = augment_and_save_samples(METHOD_2, filename="babylm_augment_m2_samples", kwargs=m2_args) # For sampling inputs to test different prompts
tmp[0]

100%|██████████| 10/10 [02:43<00:00, 16.34s/it]


{'source': 'data/babylm_data/babylm_100M/wikipedia.train',
 'text': 'The most famous hurricane in Canadian history struck on October 15, 1954, causing catastrophic flooding. Hurricane Hazel submerged low-lying land from Etobicoke to the Holland Marsh and left 81 people dead. No natural disaster since has led to such a high death toll in Canada. Over 4,000 families were left homeless.',
 'emotion': 'sadness',
 'prompt': "[INST]You are a creative writer who can write an emotional story. Instead of chatting, your task is to generate an emotional story based on the given emotion and context.\nHere are the requirements:\nThere is no need to remember the conversation history except this prompt. The history prompts are independent.\nEmotion is given as one word and context, given as a sentence or a part of sentence, follows.\nYour response is a monologue by a girl.\nFor example:\njoy: I am going on a vacation => All of a sudden, I decided that it was time for a change, a breath of fresh air. 

In [18]:
tmp = augment_and_save_samples(METHOD_3, filename="babylm_augment_m3_samples") # For sampling inputs to test different prompts
tmp[0]

100%|██████████| 10/10 [02:28<00:00, 14.82s/it]


{'source': 'data/babylm_data/babylm_100M/wikipedia.train',
 'text': 'The most famous hurricane in Canadian history struck on October 15, 1954, causing catastrophic flooding. Hurricane Hazel submerged low-lying land from Etobicoke to the Holland Marsh and left 81 people dead. No natural disaster since has led to such a high death toll in Canada. Over 4,000 families were left homeless.',
 'emotion': 'sadness',
 'prompt': "[INST]\nObjective:\nCraft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children.\nGuidelines:\nUse simple, child-friendly language throughout the story, making it easy children to understand and connect with.Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.\nStart your story creatively, avoiding traditional opening lines to engage the reader immediately.\nWhen the story involves sad or negative emotions, addr

In [ ]:
# from itertools import product

# grid = {
#     'max_new_tokens': [256],
#     'temperature': [0.7, 0.8, 0.9, 1.0, 1.1, 1.2],
#     'top_p': [1.0],
# }

# def grid_seach_augment(grid, chunk_ids=[0], batch_size=32, filename="babylm_augment", debug=False):
#     keys, values = zip(*grid.items())
#     param_combinations = [dict(zip(keys, v)) for v in product(*values)]

#     for kwargs in param_combinations:
#         kwargs = build_mistral_kwargs(kwargs)
#         print("Current arguments:", kwargs)
#         full_filename = filename + "_" + "-".join([str(v) for v in kwargs.values()])
#         augment_and_save_chunks(chunk_ids=chunk_ids, batch_size=batch_size, filename=full_filename, debug=debug, kwargs=kwargs)